# 理解程式碼內容與生成程式碼

使用 gpt-35-turbo 與 gpt-3 支援的 Completion API 來進行自動完成

In [1]:
import os
import openai
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()

# 設定呼叫 OpenAI API 所需連線資訊
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "發生錯誤: 缺少 Azure OpenAI Service 鍵值"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")
assert RESOURCE_ENDPOINT, "發生錯誤: 缺少 Azure OpenAI Service 連線端點"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "發生錯誤: Azure OpenAI Service 連線端點格式應該如: \n\n\t<您所建立之資源名稱>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model = os.getenv("DEPLOYMENT_NAME")

# 呼叫自動完成 API
response = openai.Completion.create(
  engine= model,
  prompt="### Postgres SQL tables, with their properties:\n" \
         "#\n" \
         "# Employee(id, name, department_id) \n" \
         "# Department(id, name, address)\n" \
         "# Salary_Payments(id, employee_id, amount, date) \n" \
         "### " \
         "查詢列出過去 3 個月僱用超過 10 名員工的部門名稱 \n"
         "query: SELECT",
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])

print(response.choices[0].text)

 Department.name FROM Department 
        JOIN Employee ON Department.id = Employee.department_id 
        JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id 
        WHERE Salary_Payments.date >= NOW() - INTERVAL '3 months' 
        GROUP BY Department.name 
        HAVING COUNT(DISTINCT Employee.id) > 10


嘗試使用 gpt-35-turbo 與 gpt-4 方支援的 ChatCompletion API 來進行自動完成

In [2]:
response  = openai.ChatCompletion.create( 
  engine = model ,  
  messages = [ 
    {'role': 'user', 'content': "告訴我以下這個 SQL 是什麼意思??\n"
      'Code:' \
      "SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n" \
      'Answer:'    
    }
  ],
  temperature=0,
  max_tokens=250
)

print(response['choices'][0]['message']['content']) 

這個 SQL 查詢的意思是：從 Department 表格和 Employee 表格中選擇部門名稱（d.name），並且只選擇那些在過去三個月中有薪資支付記錄的員工所在的部門。然後按照部門名稱進行分組，只顯示那些員工數量超過 10 人的部門。
